In [1]:
import camelot
import json
import os
import re

In [2]:
def get_continued_tables(tables, threshold):

    continued_tables = {}
    previous_table = False
    group_counter = 0

    # typical height of a pdf is 842 points and bottom margins are anywhere between 56 and 85 points
    # therefore, accounting for margins, 792
    page_height = 792

    # iterate over the tables
    for i, table in enumerate(tables):

        # if a previous table exists (remember, we start with this as false)
        # and the previous table was on the previous page
        # and the number of columns of both tables is the same
        if previous_table and table.page == previous_table.page + 1 and len(table.cols) == len(previous_table.cols):

            # get the bottom coordinate of the previous table
            # note that for pdfs the origin (0, 0) typically starts from the bottom-left corner of the page,
            # with the y-coordinate increasing as you move upwards
            # this is why for {x0, y0, x1, y1} we need the y0 as the bottom
            previous_table_bottom = previous_table._bbox[1]

            # get the top coordinate of the current table
            # for {x0, y0, x1, y1} we need the y1 as the top
            current_table_top = table._bbox[3]

            # if the previous table ends in the last 15% of the page and the current table starts in the first 15% of the page
            if previous_table_bottom < (threshold / 100) * page_height and current_table_top > (1 - threshold / 100) * page_height:

                # if we don't have started this group of tables
                if (continued_tables.get(group_counter) is None):

                    # start by adding the first table
                    continued_tables[group_counter] = [previous_table]

                # add any of the sunsequent tables to the group
                continued_tables[group_counter].append(table)

            # if this is not a continuation of the previous table
            else:

                # increment the group number
                group_counter += 1;

        # if this is not a continuation of the previous table
        else:

            # increment the group number
            group_counter += 1;

        # the current table becomes the previous table for the next iteration
        previous_table = table

    # transform the dictionary into an array of arrays
    continued_tables = [value for value in continued_tables.values()]

    # return the combined tables
    return continued_tables

In [3]:
def table_to_json(table_data, table_info):
    """Convert table data to JSON format"""
    if not table_data:
        return {}
    
    # Create JSON structure
    json_data = {
        "metadata": {
            "source_file": table_info["source_file"],
            "page": table_info["page"],
            "table_order": table_info["order"],
            "total_rows": len(table_data),
            "total_columns": len(table_data[0]) if table_data else 0
        },
        "headers": [],
        "data": []
    }
    
    # Add headers (first row)
    if len(table_data) > 0:
        headers = [str(cell).strip() for cell in table_data[0]]
        
        # Replace first 3 headers with fixed names
        if len(headers) >= 1:
            headers[0] = "STT"
        if len(headers) >= 2:
            headers[1] = "hang_hoa"
        if len(headers) >= 3:
            headers[2] = "yeu_cau_ky_thuat"
            
        json_data["headers"] = headers
        
        # Add data rows (skip header)
        for i, row in enumerate(table_data[1:], 1):
            row_dict = {}
            for j, cell in enumerate(row):
                # Use header as key, fallback to column index if header is empty
                key = json_data["headers"][j] if j < len(json_data["headers"]) and json_data["headers"][j] else f"column_{j}"
                row_dict[key] = str(cell).strip()
            
            json_data["data"].append({
                "row_index": i,
                "values": row_dict
            })
    
    return json_data

In [4]:
def get_biggest_table(pdf_path, threshold):
    tables = camelot.read_pdf(pdf_path, flavor = 'lattice', pages = 'all')
    continued_tables = get_continued_tables(tables, threshold)

    # get the name of the PDF file we are processing (without the extension)
    pdf_file_name = os.path.splitext(os.path.basename(pdf_path))[0]

    processed = []
    all_table_jsons = []

    # iterate over found tables
    for i, table in enumerate(tables):

        # if table was already processed as part of a group
        if table in processed: continue

        # check if the current table is a continued table
        is_continued = any(table in sublist for sublist in continued_tables)

        # collect all table data (current table + continued tables if any)
        all_table_data = list(table.data)

        # if the current table is a continued table, append all subsequent continued tables data
        if is_continued:

            # get the index of the group in "continued_tables" associated with the current table
            group_index = next(index for index, sublist in enumerate(continued_tables) if table in sublist)

            # iterate over the tables in said group and append their data
            for continued_table in continued_tables[group_index]:

                # skip the current table as it's already added
                if continued_table == table or continued_table in processed: continue

                # append the data of the continued table (skip header for subsequent tables)
                all_table_data.extend(continued_table.data[1:] if len(continued_table.data) > 1 else [])

                # keep track of processed tables
                processed.append(continued_table)

        # convert to JSON
        table_info = {
            "source_file": pdf_file_name,
            "page": table.parsing_report['page'],
            "order": table.parsing_report['order']
        }
        
        json_data = table_to_json(all_table_data, table_info)
        all_table_jsons.append(json_data)
        
        # mark current table as processed
        processed.append(table)

    # find the table with the most rows
    if all_table_jsons:
        largest_table = max(all_table_jsons, key=lambda x: x.get('metadata', {}).get('total_rows', 0))
        
        # return the JSON of the largest table
        print(json.dumps(largest_table, ensure_ascii=False, indent=2))
        return largest_table
    else:
        print("No tables found in the PDF.")
        return None

In [6]:
hello = get_biggest_table("D:/study/LammaIndex/documents/test.pdf",50)

{
  "metadata": {
    "source_file": "test",
    "page": 1,
    "table_order": 1,
    "total_rows": 15,
    "total_columns": 3
  },
  "headers": [
    "STT",
    "hang_hoa",
    "yeu_cau_ky_thuat"
  ],
  "data": [
    {
      "row_index": 1,
      "values": {
        "STT": "I",
        "hang_hoa": "Bộ chuyển đổi nguồn 220VAC/ \n48VDC (kèm theo 02 dàn acquy \n200Ah)",
        "yeu_cau_ky_thuat": ""
      }
    },
    {
      "row_index": 2,
      "values": {
        "STT": "1",
        "hang_hoa": "Yêu cầu chung",
        "yeu_cau_ky_thuat": "-  Các loại thiết bị, vật tư, phụ kiện phải có nguồn gốc xuất xứ rõ ràng, có chứng nhận chất lượng sản \nphẩm của nhà sản xuất. \n-  Thiết bị mới 100% chưa qua sử dụng \n-  Thiết bị phải được sản xuất từ năm 2021 trở lại đây \n-  Tuân thủ tiêu chuẩn IEC 60950-1 \n-  Thuộc loại thiết bị nguồn sử dụng kỹ thuật chuyển mạch, thiết kế dạng Module theo chức năng của \ntừng khối. \n-  Tất  cả  các  khối  đặt  trong  tủ  liên  hoàn,  đồng  bộ của  hãng,  

In [6]:
data = hello["data"]

In [7]:
import uuid
def clean_text(text):
    """Làm sạch text, loại bỏ ký tự xuống dòng thừa"""
    return re.sub(r'\n+', '', text.strip())

def split_requirements(text):
    """Tách các yêu cầu dựa trên dấu gạch đầu dòng"""
    requirements = []
    lines = text.split('\n')
    for line in lines:
        line = line.strip()
        if line.startswith('- '):
            requirements.append(line[2:].strip())
        elif line and not any(line.startswith(prefix) for prefix in ['- ']):
            if requirements:
                requirements[-1] += ' ' + line
            else:
                requirements.append(line)
    return requirements

def generate_random_key():
    """Tạo key random 5 ký tự từ UUID"""
    return str(uuid.uuid4()).replace('-', '')[:5].upper()

def convert_to_new_format(data):
    result = []
    current_product = None
    current_category = None
    
    for item in data:
        values = item['values']
        stt_raw  = values['STT']
        hang_hoa = clean_text(values['hang_hoa'])
        yeu_cau = values['yeu_cau_ky_thuat']


        stt = stt_raw.strip()

        roman_pattern = r'^(VII|VIII|IX|X|XI|XII|I{1,3}|IV|V|VI)\s+(.+)'
        roman_match = re.match(roman_pattern, stt)
        # Nếu STT là số La Mã (I, II, III...) thì đây là tên sản phẩm
        hang_hoa_roman_match = re.match(roman_pattern, hang_hoa)
        if roman_match and not hang_hoa and not yeu_cau:
            if current_product:
                result.append(current_product)
            
            roman_num = roman_match.group(1)  # Số La Mã
            product_name = roman_match.group(2)  # Tên sản phẩm
            
            current_product = {
                "ten_san_pham": product_name,
                "cac_muc": []
            }
            current_category = None
        elif hang_hoa_roman_match and not stt_raw and not yeu_cau:
            if current_product:
                result.append(current_product)
            
            roman_num = hang_hoa_roman_match.group(1)  # Số La Mã
            product_name = hang_hoa_roman_match.group(2)  # Tên sản phẩm
            
            current_product = {
                "ten_san_pham": product_name,
                "cac_muc": []
            }
            current_category = None        
        
        elif stt in ['I', 'II', 'III', 'IV', 'V', 'VI', 'VII', 'VIII', 'IX', 'X', 'XI', 'XII', 'XIII', 'XIV']:
            if current_product:
                result.append(current_product)
            
            current_product = {
                "ten_san_pham": hang_hoa,
                "cac_muc": []
            }
            current_category = None
            
        # Nếu STT là số (1, 2, 3...) thì đây là danh mục
        elif stt.isdigit():
            current_category = {
                "ten_hang_hoa": hang_hoa,
                "thong_so_ky_thuat": {}
            }
            
            # Xử lý yêu cầu kỹ thuật cho danh mục
            if yeu_cau.strip():
                requirements = split_requirements(yeu_cau)
                for req in requirements:
                    key = generate_random_key()  # Tạo key random
                    current_category["thong_so_ky_thuat"][key] = clean_text(req)
            if current_product:
                current_product["cac_muc"].append(current_category)
                
        # Nếu STT trống thì đây là thông số kỹ thuật chi tiết
        elif stt == '' and current_category and hang_hoa:
            # Tạo key random cho thông số kỹ thuật
            key = generate_random_key()
            
            # Làm sạch tên hàng hóa và yêu cầu kỹ thuật
            clean_hang_hoa = clean_text(hang_hoa)
            clean_yeu_cau = clean_text(yeu_cau)
            
            current_category["thong_so_ky_thuat"][key] = [clean_hang_hoa, clean_yeu_cau]
        elif stt == '' and current_category and not hang_hoa:
            if yeu_cau.strip():
                requirements = split_requirements(yeu_cau)
                
                # Lấy key cuối cùng trong thong_so_ky_thuat (nếu có)
                existing_keys = list(current_category["thong_so_ky_thuat"].keys())
                last_key = existing_keys[-1] if existing_keys else None
                
                for req in requirements:
                    clean_req = clean_text(req)
                    
                    # Kiểm tra chữ cái đầu có viết hoa HOẶC có gạch đầu dòng không
                    has_dash = req.strip().startswith('- ')
                    has_uppercase = clean_req and clean_req[0].isupper()
                    
                    if has_uppercase or has_dash:
                        # Chữ đầu viết hoa HOẶC có gạch đầu dòng -> tạo key mới
                        key = generate_random_key()
                        current_category["thong_so_ky_thuat"][key] = clean_req
                        last_key = key
                    else:
                        # Chữ đầu không viết hoa VÀ không có gạch đầu dòng -> nối vào key trước đó
                        if last_key and last_key in current_category["thong_so_ky_thuat"]:
                            current_category["thong_so_ky_thuat"][last_key] += " " + clean_req
                        else:
                            # Nếu không có key trước đó thì vẫn tạo key mới
                            key = generate_random_key()
                            current_category["thong_so_ky_thuat"][key] = clean_req
                            last_key = key
    
    # Thêm sản phẩm cuối cùng
    if current_product:
        result.append(current_product)
    
    return result

# Chuyển đổi dữ liệu
converted_data = convert_to_new_format(data)

In [8]:
converted_data

[{'ten_san_pham': 'Tải giả xả acquy',
  'cac_muc': [{'ten_hang_hoa': 'Yêu cầu chung',
    'thong_so_ky_thuat': {'5AEBD': 'Các loại thiết bị, vật tư, phụ kiện phải có nguồn gốc xuất xứ, có chứng nhận chất lượng sản phẩm của nhà sản xuất.',
     '26659': 'Thiết bị mới 100% chưa qua sử dụng',
     '5E9BA': 'Thiết bị phải được sản xuất từ năm 2021 trở lại đây',
     '60BF4': 'Thời  gian  bảo  hành:  theo  tiêu  chuẩn  của  nhà  sản xuất, tối thiểu 12 tháng.'}},
   {'ten_hang_hoa': 'Thông số kỹ thuật',
    'thong_so_ky_thuat': {'643D2': ['Điện trở tiếp địa', '< 0.1 Ohm'],
     'E5FDF': ['Độ ồn', '< 75 dB khi hoạt động hết công suất.'],
     'B33FC': ['Điều kiện nhiệt độ',
      '-  Dải nhiệt độ hoạt động: 0 ÷ 40°C,  -  Dải nhiệt độ lưu kho: -25°C ÷70°C.'],
     '48365': ['Điều kiện độ ẩm',
      '-  Độ ẩm tương đối từ 5 ÷ 95%, không đọng sương.'],
     '6274F': ['Điện áp acquy cao nhất – tính theo giá trị tuyệt đối',
      '≥ 270 V'],
     'E5C39': ['Điện áp acquy nhỏ nhất', '≤ 18 V'],
    

In [9]:
context_queries = {}  # Dict chứa thông tin chi tiết theo key
product_key = {}  # Dict lồng: ten_san_pham -> ten_hang_hoa -> list[key]

for item in converted_data:
    ten_san_pham = item['ten_san_pham']
    for muc in item['cac_muc']:
        ten_hang_hoa = muc['ten_hang_hoa']
        thong_so_ky_thuat = muc['thong_so_ky_thuat']
        for key, value in thong_so_ky_thuat.items():
            if isinstance(value, list):
                q = value[0]
                k = value[1]
                value_str = ' '.join(value)
            else:
                q = None
                k = value
                value_str = value

            # Ghi vào context_queries
            context_queries[key] = {
                "ten_san_pham": ten_san_pham,
                "ten_hang_hoa": ten_hang_hoa,
                "value": value_str,
                "yeu_cau_ky_thuat_chi_tiet": k,
                "yeu_cau_ky_thuat": q
            }

            # Ghi vào product_key
            if ten_san_pham not in product_key:
                product_key[ten_san_pham] = {}
            if ten_hang_hoa not in product_key[ten_san_pham]:
                product_key[ten_san_pham][ten_hang_hoa] = []
            product_key[ten_san_pham][ten_hang_hoa].append(key)



In [ ]:
context_queries

In [ ]:
product_key

In [ ]:
# Lưu với tên file mặc định
with open('context_prompts.json', 'w', encoding='utf-8') as f:
    json.dump(context_prompts, f, ensure_ascii=False, indent=2)

print(f"✅ Đã lưu {len(context_prompts)} context prompts vào file: context_prompts.json")

In [10]:
def prompt_create_query(context_prompt):
    prompt =  f"""
    Bạn sẽ nhận được một đoạn mô tả kỹ thuật ngắn gọn (context_prompt), thường là các mảnh thông tin kỹ thuật rời rạc. Nhiệm vụ của bạn là chuyển đổi đoạn mô tả đó thành một truy vấn (query) hoàn chỉnh bằng ngôn ngữ tự nhiên, với mục tiêu:

- Diễn đạt lại đoạn mô tả dưới dạng một câu hỏi hoặc câu truy vấn rõ ràng, dễ hiểu.
- Giữ nguyên đầy đủ tất cả các thông tin kỹ thuật có trong câu gốc (chế độ, dòng điện, điện áp, bước công suất,...).
- Không được loại bỏ hoặc làm mờ bất kỳ chi tiết kỹ thuật nào.
- Truy vấn cần phù hợp để tìm kiếm thông tin tương đồng về mặt ngữ nghĩa trong một hệ thống truy xuất dữ liệu kỹ thuật.

Dưới đây là các ví dụ:

**Input:** "Tải giả xả acquy Thông số kỹ thuật Dòng xả lớn nhất với dải điện áp 220V DC ÷ 240V DC – tương ứng với 1 bộ ≥ 70 A"

**Output:** "Dòng xả lớn nhất của tải giả xả acquy là bao nhiêu khi dải điện áp nằm trong khoảng 220V DC đến 240V DC, và tương ứng với 1 bộ thì có đạt dòng xả ≥ 70A không?"

---
**Input:** "Tải giả xả acquy Thông số kỹ thuật Các chế độ xả - Gồm 4 chế độ: dòng không đổi, công suất không đổi, theo đặc tính dòng cho trước, điều chỉnh thủ công."

**Output:** "Tải giả xả acquy hỗ trợ những chế độ xả nào? Có phải gồm dòng không đổi, công suất không đổi, theo đặc tính dòng định trước, và điều chỉnh thủ công không?"

---
**Input:** "Tải giả xả acquy Thông số kỹ thuật Bước công suất điều chỉnh xả tải 100 W"

**Output:** "Bước điều chỉnh công suất xả tải nhỏ nhất của tải giả xả acquy là bao nhiêu? Có phải là 100W không?"

---
Trả lời từng `context_prompt` theo cấu trúc như trên.


    DỮ LIỆU ĐẦU VÀO:
    ---
    {context_prompt}
    ---

    CHỈ CẦN TRẢ VỀ CÂU TRUY VẤN.
    """
    return prompt

In [11]:
from openai import OpenAI
from dotenv import load_dotenv
import re
import os
load_dotenv()

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [12]:
test = dict(list(context_queries.items())[71:99])
test

{'96103': {'ten_san_pham': 'Bộ chuyển đổi nguồn 220VAC/ 48VDC (kèm theo 02 dàn acquy 200Ah)',
  'ten_hang_hoa': 'Cấu hình thiết bị nguồn',
  'value': 'Số lượng khe cắm module chỉnh lưu (Rectifier): ≥ 4',
  'yeu_cau_ky_thuat_chi_tiet': 'Số lượng khe cắm module chỉnh lưu (Rectifier): ≥ 4',
  'yeu_cau_ky_thuat': None},
 '277B7': {'ten_san_pham': 'Bộ chuyển đổi nguồn 220VAC/ 48VDC (kèm theo 02 dàn acquy 200Ah)',
  'ten_hang_hoa': 'Cấu hình thiết bị nguồn',
  'value': 'Công suất mỗi module chỉnh lưu: ≥ 3000W',
  'yeu_cau_ky_thuat_chi_tiet': 'Công suất mỗi module chỉnh lưu: ≥ 3000W',
  'yeu_cau_ky_thuat': None},
 '794FC': {'ten_san_pham': 'Bộ chuyển đổi nguồn 220VAC/ 48VDC (kèm theo 02 dàn acquy 200Ah)',
  'ten_hang_hoa': 'Cấu hình thiết bị nguồn',
  'value': 'Số lượng module chỉnh lưu trang bị kèm tủ nguồn: ≥ 3 module',
  'yeu_cau_ky_thuat_chi_tiet': 'Số lượng module chỉnh lưu trang bị kèm tủ nguồn: ≥ 3 module',
  'yeu_cau_ky_thuat': None},
 '3ACE9': {'ten_san_pham': 'Bộ chuyển đổi nguồn 22

In [13]:
for key in test:
    data = test[key] 
    prompt = prompt_create_query(f"{data['ten_san_pham']} {data['ten_hang_hoa']} {data['value']}")
    response = client.responses.create(
        model="gpt-4o-mini",
        input=prompt,
        temperature=0
    )
    output_text = response.output_text.strip()
    test[key]["query"]=output_text

In [ ]:
test, product_key

In [14]:
import os
from dotenv import load_dotenv
from llama_index.core import VectorStoreIndex
from llama_index.core.vector_stores import VectorStoreInfo
from llama_index.vector_stores.qdrant import QdrantVectorStore
from qdrant_client import QdrantClient, AsyncQdrantClient
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core.retrievers import VectorIndexAutoRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.settings import Settings
from llama_index.core.vector_stores import (
    MetadataFilter,
    MetadataFilters,
    FilterOperator,
    FilterCondition
)

# Cấu hình LLM và Embedding
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")
Settings.llm = OpenAI(model="gpt-4o-mini")
# Cấu hình client Qdrant
client = QdrantClient(
    url="https://a8bcf78f-0147-411f-aa58-079f863fcd6d.us-west-1-0.aws.cloud.qdrant.io:6333",
    api_key=os.getenv("QDRANT_API_KEY"),
)
aclient = AsyncQdrantClient(
    url="https://a8bcf78f-0147-411f-aa58-079f863fcd6d.us-west-1-0.aws.cloud.qdrant.io:6333",
    api_key=os.getenv("QDRANT_API_KEY"),
)
# Khởi tạo Vector Store
vector_store = QdrantVectorStore(
    collection_name="thong_tin_san_pham",
    client=client,
    aclient=aclient,
)

def retrieve_document(query_str):
    file_names = []
    index = VectorStoreIndex.from_vector_store(vector_store=vector_store)
    
    filters_document = MetadataFilters(
        filters=[
            MetadataFilter(key="type", operator=FilterOperator.EQ, value="summary_document"),
        ],
    condition=FilterCondition.AND,
    )
    retriever_document = index.as_retriever(similarity_top_k=5, verbose=True, filters=filters_document)
    
    results = retriever_document.retrieve(query_str)

    for result in results:
        metadata = result.metadata
        file_names.append(metadata["file_name"])

    return file_names

def retrieve_chunk(file_names, query_str):
    index = VectorStoreIndex.from_vector_store(vector_store=vector_store)
    
    filters_chunk = MetadataFilters(
        filters=[
            MetadataFilter(key="file_name", operator=FilterOperator.IN, value=file_names),
            MetadataFilter(key="type", operator=FilterOperator.EQ, value="chunk_document"),
        ],
        condition=FilterCondition.AND,
    )

    retriever_chunk = index.as_retriever(similarity_top_k=5, verbose=True, filters=filters_chunk)
    
    results = retriever_chunk.retrieve(query_str)
    content = ""
    for i, result in enumerate(results, start=1):
        metadata = result.metadata
        file_name = metadata["file_name"]+ ".pdf"
        page = metadata["page"]
        table = metadata["table_name"]
        figure_name = metadata.get("figure_name")
        text = result.text.strip()
        content += f"Chunk {i} trong file {file_name} tại trang {page}, có chứa bảng {table} và hình {figure_name} có nội dung:\n{text}\n\n"

    return content


        




D:\study\LammaIndex\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [59]:
for product in product_key:
    file_names = retrieve_document(product)
    print(file_names)
    items = product_key[product]
    for key in items:
        for item in items[key]:
            if item not in test:
                continue
            query = test[item]["query"]
            content = retrieve_chunk(file_names, query)
            test[item]["content"] = content            
            

['vertiv-esure-inverter-i120-1000-ds-en-gl-dc', 'NetSure_732_User_Manual', 'NetSure_732_Brochure', 'Converter_Brochure', 'Controller_Brochure']
['Converter_Brochure', 'NetSure_732_Brochure', 'Controller_Brochure', 'vertiv-esure-inverter-i120-1000-ds-en-gl-dc', 'NetSure_732_User_Manual']
['Converter_Brochure', 'NetSure_732_User_Manual', 'vertiv-esure-inverter-i120-1000-ds-en-gl-dc', 'NetSure_732_Brochure', 'Controller_Brochure']
['Converter_Brochure', 'vertiv-esure-inverter-i120-1000-ds-en-gl-dc', 'Controller_Brochure', 'NetSure_732_Brochure', 'NetSure_732_User_Manual']
['NetSure_732_Brochure', 'Controller_Brochure', 'NetSure_732_User_Manual', 'Converter_Brochure', 'vertiv-esure-inverter-i120-1000-ds-en-gl-dc']
['Converter_Brochure', 'NetSure_732_User_Manual', 'vertiv-esure-inverter-i120-1000-ds-en-gl-dc', 'NetSure_732_Brochure', 'Controller_Brochure']
['NetSure_732_User_Manual', 'NetSure_732_Brochure', 'vertiv-esure-inverter-i120-1000-ds-en-gl-dc', 'Converter_Brochure', 'Controller_Bro

In [16]:
for key in test:
    print(key)

96103
277B7
794FC
3ACE9
992F3
E90A0
26AB9
0F7D2
5C1A5
09D89
4731B
2188A
656E3
F916B
8502B
78D05
D0AA4
90A03
66172
88C14
A1813
B5749
4CC2F
0EE91
0D76B
0649F
F74FE
D701F


In [17]:
SYSTEM_PROMPT = """
Bạn được cung cấp:
- Một hoặc nhiều đoạn văn bản (chunk) từ tài liệu kỹ thuật, kèm metadata: tên file, mục, bảng/hình (nếu có), số trang
- Một yêu cầu kỹ thuật cụ thể.
- Một đoạn văn mẫu.

#Yêu cầu trả lời bằng tiếng việt:
# 1. Tìm thông tin kỹ thuật liên quan trực tiếp đến yêu cầu kỹ thuật.
# 2. Trích xuất giá trị thông số để xác định khả năng đáp ứng theo yêu cầu và trả về đoạn văn tương tự giống đoạn văn mẫu không thêm bớt nhưng thông số phải chính xác trong tài liệu.
# 3. Dẫn chứng rõ: file, section, table/figure name (nếu có), page, nội dung trích dẫn của những tài liệu liên quan, những tài liệu khác không liên quan thì bỏ qua.

# #Output: JSON gồm các trường:
- yeu_cau_ky_thuat
- kha_nang_dap_ung
- tai_lieu_tham_chieu" 

# Ví dụ:
Input:
Yêu cầu: "Số lượng khe cắm module chỉnh lưu (Rectifier): ≥ 4"  
Chunk: "...NetSure 731 A41-S8: 4 rectifier slots (standard), expandable to 6..."  
Metadata:  
- file: "Netsure-731-A41-user-manual.pdf"  
- section: "Table 1-1 Configuration of power system"  
- page: 2"
Đoạn văn mẫu: Số lượng khe cắm module chỉnh lưu (Rectifier): ≥ 4 (ví dụ tìm trong tài liệu số lượng là 5 thì trả về "Số lượng khe cắm module chỉnh lưu (Rectifier): 5")
"""

# Định nghĩa function schema
FUNCTION_SCHEMA = {
    "name": "danh_gia_ky_thuat",
    "description": "Đánh giá khả năng đáp ứng của sản phẩm theo yêu cầu kỹ thuật từ chunk tài liệu.",
    "parameters": {
        "type": "object",
        "properties": {
            "yeu_cau_ky_thuat": {"type": "string"},
            "kha_nang_dap_ung": {"type": "string"},
            "tai_lieu_tham_chieu": {
                "type": "object",
                "properties": {
                    "file": {"type": "string"},
                    "section": {"type": "string"},
                    "table_or_figure": {"type": "string"},
                    "page": {"type": "integer"},
                    "evidence": {"type": "string"}
                },
                "required": ["file", "section", "page", "evidence"]
            }
        },
        "required": ["yeu_cau_ky_thuat", "kha_nang_dap_ung", "tai_lieu_tham_chieu"]
    }
}

In [18]:
# Tạo assistant
# === CREATE ASSISTANT ===
def create_assistant():
    assistant = client.beta.assistants.create(
        name="Technical Document Evaluator",
        instructions=SYSTEM_PROMPT,
        model="gpt-4o-mini",
        tools=[{"type": "function", "function": FUNCTION_SCHEMA}]
    )
    return assistant.id

# === CREATE THREAD ===
def create_thread():
    thread = client.beta.threads.create()
    return thread.id

# === UPDATE ASSISTANT ===
def update_assistant(assistant_id):
    assistant = client.beta.assistants.update(
        assistant_id=assistant_id,
        instructions=SYSTEM_PROMPT,
        model="gpt-4o-mini",
        tools=[{"type": "function", "function": FUNCTION_SCHEMA}]
    )
    return assistant.id

# === EVALUATE TECHNICAL REQUIREMENT ===
def evaluate_technical_requirement(user_prompt, assistant_id):
    # 1. Tạo thread riêng cho mỗi lần gọi
    thread = client.beta.threads.create()
    thread_id = thread.id

    # 2. Gửi message vào thread
    client.beta.threads.messages.create(
        thread_id=thread_id,
        role="user",
        content=user_prompt
    )

    # 3. Tạo run
    run = client.beta.threads.runs.create(
        thread_id=thread_id,
        assistant_id=assistant_id,
        tool_choice={"type": "function", "function": {"name": "danh_gia_ky_thuat"}}
    )

    # 4. Chờ assistant xử lý (tối đa 20s)
    for _ in range(20):
        run = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run.id)
        if run.status not in ["queued", "in_progress"]:
            break
        time.sleep(1)

    # 5. Lấy arguments trực tiếp
    if run.status == "requires_action":
        call = run.required_action.submit_tool_outputs.tool_calls[0]
        print(f"👉 Assistant đã gọi tool: {call.function.name}")
        print("🧠 Dữ liệu JSON assistant muốn trả về:")
        print(call.function.arguments)
        return call.function.arguments

    elif run.status == "completed":
        messages = client.beta.threads.messages.list(thread_id=thread_id)
        for msg in messages.data:
            print(f"[{msg.role}] {msg.content[0].text.value}")
        return None

    else:
        print(f"Run status: {run.status}")
        return None



In [19]:
test

{'96103': {'ten_san_pham': 'Bộ chuyển đổi nguồn 220VAC/ 48VDC (kèm theo 02 dàn acquy 200Ah)',
  'ten_hang_hoa': 'Cấu hình thiết bị nguồn',
  'value': 'Số lượng khe cắm module chỉnh lưu (Rectifier): ≥ 4',
  'yeu_cau_ky_thuat_chi_tiet': 'Số lượng khe cắm module chỉnh lưu (Rectifier): ≥ 4',
  'yeu_cau_ky_thuat': None,
  'query': '"Bộ chuyển đổi nguồn 220VAC/48VDC có cấu hình thiết bị nguồn với số lượng khe cắm module chỉnh lưu (Rectifier) là bao nhiêu? Có phải là ≥ 4 không?"',
  'content': 'Chunk 1 trong file NetSure_732_Brochure.pdf tại trang 1, có chứa bảng System Configuration of NetSure™ 732 A41 và hình None có nội dung:\n## Description\n\nNetSure™ 732 A41 is a rack-mounted power system designed to accept different\ninput voltages with the capability to power 48VDC access & core equipment of\nthe telecom sites. Built with cutting-edge technology, it comes equipped with a\nmodular and robust hot-pluggable converter and a turnkey controller module\ndesign, which ensures the highest upti

In [20]:
import openai
from openai import OpenAI
import json
client = OpenAI()

In [21]:
# Ví dụ sử dụng
import time
assistant_id = create_assistant()
print(f"Assistant ID: {assistant_id}")


# Ví dụ user prompt

for key in test:
    if not test[key].get('content'):
        continue
    data = test[key]
    value = data['value']
    content = data['content']
    form = data['yeu_cau_ky_thuat_chi_tiet']
    # test[key].pop("content", None)
    
    user_prompt = f'''
    Chunk và metadata: {content}
    Yêu cầu: {value}
    Đoạn văn mẫu: {form}
    '''
    
    # Gọi hàm đánh giá với thread riêng
    result = evaluate_technical_requirement(user_prompt, assistant_id)
    test[key]['response'] = result




Assistant ID: asst_bQbH4lkSQegK4AqUNFxtJ8lx


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_4236\985798817.py:30: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread = client.beta.threads.create()
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_4236\985798817.py:34: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  client.beta.threads.messages.create(
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_4236\985798817.py:41: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.create(
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_4236\985798817.py:49: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run.id)


👉 Assistant đã gọi tool: danh_gia_ky_thuat
🧠 Dữ liệu JSON assistant muốn trả về:
{"yeu_cau_ky_thuat":"Số lượng khe cắm module chỉnh lưu (Rectifier): ≥ 4","kha_nang_dap_ung":"Số lượng khe cắm module chỉnh lưu (Rectifier): 4","tai_lieu_tham_chieu":{"file":"NetSure_732_User_Manual.pdf","section":"Table 1-1 Configuration of power system","table_or_figure":"Table 1-1","page":8,"evidence":"Maximum configuration：4 pieces"}}
👉 Assistant đã gọi tool: danh_gia_ky_thuat
🧠 Dữ liệu JSON assistant muốn trả về:
{"yeu_cau_ky_thuat":"Công suất mỗi module chỉnh lưu: ≥ 3000W","kha_nang_dap_ung":"Công suất mỗi module chỉnh lưu: 3000W Maximum","tai_lieu_tham_chieu":{"file":"Converter_Brochure.pdf","section":"DC Output Specifications","table_or_figure":"None","page":2,"evidence":"Công suất mỗi module chỉnh lưu: 3000W Maximum"}}
👉 Assistant đã gọi tool: danh_gia_ky_thuat
🧠 Dữ liệu JSON assistant muốn trả về:
{"yeu_cau_ky_thuat":"Số lượng module chỉnh lưu trang bị kèm tủ nguồn: ≥ 3 module","kha_nang_dap_ung":

In [29]:
test

{'96103': {'ten_san_pham': 'Bộ chuyển đổi nguồn 220VAC/ 48VDC (kèm theo 02 dàn acquy 200Ah)',
  'ten_hang_hoa': 'Cấu hình thiết bị nguồn',
  'value': 'Số lượng khe cắm module chỉnh lưu (Rectifier): ≥ 4',
  'yeu_cau_ky_thuat_chi_tiet': 'Số lượng khe cắm module chỉnh lưu (Rectifier): ≥ 4',
  'yeu_cau_ky_thuat': None,
  'query': '"Bộ chuyển đổi nguồn 220VAC/48VDC có cấu hình thiết bị nguồn với số lượng khe cắm module chỉnh lưu (Rectifier) là bao nhiêu? Có phải là ≥ 4 không?"',
  'kha_nang_dap_ung': 'Số lượng khe cắm module chỉnh lưu (Rectifier): 4',
  'tai_lieu_tham_chieu': {'file': 'NetSure_732_User_Manual.pdf',
   'section': 'Table 1-1 Configuration of power system',
   'table_or_figure': 'Table 1-1',
   'page': 8,
   'evidence': 'Maximum configuration：4 pieces'}},
 '277B7': {'ten_san_pham': 'Bộ chuyển đổi nguồn 220VAC/ 48VDC (kèm theo 02 dàn acquy 200Ah)',
  'ten_hang_hoa': 'Cấu hình thiết bị nguồn',
  'value': 'Công suất mỗi module chỉnh lưu: ≥ 3000W',
  'yeu_cau_ky_thuat_chi_tiet': '

In [ ]:
import json

for key in test:
    test[key].pop("content", None)
    response = test[key].get('response')  # Sử dụng get để tránh lỗi nếu không có response
    print(response)
    if response:
        if isinstance(response, str):
            first_json_str = response[:response.find('}}') + 2]
            response = json.loads(first_json_str)
        test[key]["kha_nang_dap_ung"] = response.get('kha_nang_dap_ung', '')
        tai_lieu_tham_chieu = {
            "file": response['tai_lieu_tham_chieu'].get('file', ''),
            "section": response['tai_lieu_tham_chieu'].get('section', ''),
            "table_or_figure": response['tai_lieu_tham_chieu'].get('table_or_figure', ''),
            "page": response['tai_lieu_tham_chieu'].get('page', 0),
            "evidence": response['tai_lieu_tham_chieu'].get('evidence', '')
        }
        test[key]["tai_lieu_tham_chieu"] = tai_lieu_tham_chieu
    test[key].pop("response", None)


In [44]:
def prompt_adapt_or_not(dap_ung_ky_thuat: str) -> str:
    prompt = f"""
Bạn sẽ được cung cấp một danh sách các cặp “yêu cầu kỹ thuật || khả năng đáp ứng” trong file dap_ung_ky_thuat.
Nhiệm vụ của bạn:
1. Với từng cặp, đánh giá xem khả năng đáp ứng có thực sự đáp ứng yêu cầu kỹ thuật không.
2. Tổng hợp kết quả:
  – Nếu tất cả các yêu cầu đều được đáp ứng, trả về "Đáp ứng"
  – Nếu không có yêu cầu nào được đáp ứng, trả về "Không đáp ứng"
  – Nếu chỉ một phần yêu cầu được đáp ứng, trả về theo định dạng "Đáp ứng: x/y = z%", trong đó:
       - x là số yêu cầu được đáp ứng
       - y là tổng số yêu cầu
       - z% là phần trăm làm tròn đến số nguyên gần nhất
📤 Kết quả chỉ trả về dưới dạng JSON với cấu trúc sau: {{
 "đáp ứng kỹ thuật": "<kết quả đánh giá>"
}}

Danh sách các cặp “yêu cầu kỹ thuật || khả năng đáp ứng” : {dap_ung_ky_thuat}
"""
    return prompt

In [36]:
from openai import OpenAI
from dotenv import load_dotenv
import re
import os
load_dotenv()

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [45]:
def parse_output_text(output_text: str) -> dict:
    # B1: Loại bỏ phần ```json ... ```
    cleaned = re.sub(r"^```json\n|```$", "", output_text.strip())
    print(cleaned)
    # B2: Giải mã các ký tự escape như \n, \"
    unescaped = cleaned.encode("utf-8")

    # B3: Chuyển thành dict
    return json.loads(unescaped)

In [46]:
results = []
for product in product_key:
    items = product_key[product]
    for key in items:
        dap_ung_ky_thuat = ""
        tai_lieu_tham_chieu = ""
        for item in items[key]:
            if item not in test:
                continue
            yeu_cau_ky_thuat = test[item].get('yeu_cau_ky_thuat_chi_tiet', "")
            kha_nang_dap_ung = test[item].get('kha_nang_dap_ung', '')
            dap_ung_ky_thuat += f"{yeu_cau_ky_thuat} || {kha_nang_dap_ung}\n"
            
            tai_lieu = test[item].get('tai_lieu_tham_chieu', {})
            file = tai_lieu.get("file", "")
            page = tai_lieu.get("page", "")
            table_or_figure = tai_lieu.get("table_or_figure", "")
            evidence = tai_lieu.get("evidence", "")
            
            # Tạo câu mô tả tài liệu
            tai_lieu_text = f"{file}, trang: {page}"
            if table_or_figure:
                tai_lieu_text += f", trong bảng(figure): {table_or_figure}"
            tai_lieu_text += f", evidence: {evidence}\n\n"
            tai_lieu_tham_chieu += tai_lieu_text  # 🔧 thiếu dòng này

        if dap_ung_ky_thuat and tai_lieu_tham_chieu:  # Có thể dùng trực tiếp như bool
            prompt = prompt_adapt_or_not(dap_ung_ky_thuat)
            response = client.responses.create(
                model="gpt-4o-mini",
                input=prompt,
                temperature=0
            )
            output_text = response.output_text.strip()
            output_text = parse_output_text(output_text)
            print(output_text)
            product_key[product][key].append(output_text['đáp ứng kỹ thuật'])
            product_key[product][key].append(tai_lieu_tham_chieu)


{
  "đáp ứng kỹ thuật": "Đáp ứng: 5/5 = 100%"
}

{'đáp ứng kỹ thuật': 'Đáp ứng: 5/5 = 100%'}
{
  "đáp ứng kỹ thuật": "Đáp ứng"
}

{'đáp ứng kỹ thuật': 'Đáp ứng'}
{
  "đáp ứng kỹ thuật": "Đáp ứng: 3/5 = 60%"
}

{'đáp ứng kỹ thuật': 'Đáp ứng: 3/5 = 60%'}
{
  "đáp ứng kỹ thuật": "Đáp ứng: 7/8 = 88%"
}

{'đáp ứng kỹ thuật': 'Đáp ứng: 7/8 = 88%'}
{
  "đáp ứng kỹ thuật": "Đáp ứng: 6/7 = 86%"
}

{'đáp ứng kỹ thuật': 'Đáp ứng: 6/7 = 86%'}


In [52]:
product_key 

{'Tải giả xả acquy': {'Yêu cầu chung': ['5AEBD', '26659', '5E9BA', '60BF4'],
  'Thông số kỹ thuật': ['643D2',
   'E5FDF',
   'B33FC',
   '48365',
   '6274F',
   'E5C39',
   'D8E86',
   'F2148',
   'EF552',
   '51A2D',
   '1502C',
   'CBF00',
   '8C0C4',
   '12A1F',
   '79DEE',
   'D7415',
   '6F223',
   '15A24',
   'E04DC',
   '6C12B',
   '390FF',
   '62143',
   '2B6F4',
   'F5D0C',
   '43F09',
   '34C12',
   '5A287',
   '3DFE5',
   '0857C',
   'BB575',
   '4ACFD',
   'C0D67',
   '17B83',
   '2980A',
   'E776B',
   '0E99B',
   'FE1B1',
   'D0154',
   '45BD6',
   'F080A']},
 'Đồng hồ đo nội trở acquy': {'Yêu cầu chung': ['12141', 'E6B1E', '8094D'],
  'Các tính năng': ['D40E4', '69E7B', 'B69C5'],
  'Đặc tính kỹ thuật': ['C7451', 'C9BF6', '08A28'],
  'Thân máy': ['41DEC', '18DC2', '775D0', 'BEC9B'],
  'Pin': ['DF1C1', '717FA'],
  'Phụ kiện': ['9F7DF', '7B7C7', '3F8A9', 'D4EA1', 'ACA1C']},
 'Bộ chuyển đổi nguồn 220VAC/ 48VDC (kèm theo 02 dàn acquy 200Ah)': {'Yêu cầu chung': ['4E1C2',
   '5

In [35]:
test

{'96103': {'ten_san_pham': 'Bộ chuyển đổi nguồn 220VAC/ 48VDC (kèm theo 02 dàn acquy 200Ah)',
  'ten_hang_hoa': 'Cấu hình thiết bị nguồn',
  'value': 'Số lượng khe cắm module chỉnh lưu (Rectifier): ≥ 4',
  'yeu_cau_ky_thuat_chi_tiet': 'Số lượng khe cắm module chỉnh lưu (Rectifier): ≥ 4',
  'yeu_cau_ky_thuat': None,
  'query': '"Bộ chuyển đổi nguồn 220VAC/48VDC có cấu hình thiết bị nguồn với số lượng khe cắm module chỉnh lưu (Rectifier) là bao nhiêu? Có phải là ≥ 4 không?"',
  'kha_nang_dap_ung': 'Số lượng khe cắm module chỉnh lưu (Rectifier): 4',
  'tai_lieu_tham_chieu': {'file': 'NetSure_732_User_Manual.pdf',
   'section': 'Table 1-1 Configuration of power system',
   'table_or_figure': 'Table 1-1',
   'page': 8,
   'evidence': 'Maximum configuration：4 pieces'}},
 '277B7': {'ten_san_pham': 'Bộ chuyển đổi nguồn 220VAC/ 48VDC (kèm theo 02 dàn acquy 200Ah)',
  'ten_hang_hoa': 'Cấu hình thiết bị nguồn',
  'value': 'Công suất mỗi module chỉnh lưu: ≥ 3000W',
  'yeu_cau_ky_thuat_chi_tiet': '

In [54]:
product_key_test = {
    'Bộ chuyển đổi nguồn 220VAC/ 48VDC (kèm theo 02 dàn acquy 200Ah)':{
         'Cấu hình thiết bị nguồn': ['96103',
   '277B7',
   '794FC',
   '3ACE9',
   'Đáp ứng: 5/5 = 100%',
   'NetSure_732_User_Manual.pdf, trang: 8, trong bảng(figure): Table 1-1, evidence: Maximum configuration：4 pieces\n\nConverter_Brochure.pdf, trang: 2, trong bảng(figure): None, evidence: Công suất mỗi module chỉnh lưu: 3000W Maximum\n\nNetSure_732_User_Manual.pdf, trang: 8, evidence: Maximum configuration: 4 pieces\n\nNetSure_732_User_Manual.pdf, trang: 10, trong bảng(figure): Table 1-1 Configuration of power system, evidence: Battery MCB: 2×125A/1P; AC output MCB: 1×16A/1P; Load route rated current | Max. output current | Min. cable CSA | Max cable length (volt drop: 0.5V with min. CSA) | Max. cable CSA | Max cable length (volt drop: 0.5V with max. CSA) : 63A | 50A | 16mm2 | 9m | 25mm2 | 14m; 32A | 25A | 10mm2 | 11m | 25mm2 | 29m; 16A | 12A | 6 mm2 | 14m | 25mm2 | 48m.\n\n'],
  'Đầu vào AC': ['992F3',
   'E90A0',
   '26AB9',
   '0F7D2',
   'Đáp ứng',
   'NetSure_732_User_Manual.pdf, trang: 8, trong bảng(figure): Table 1-1, evidence: AC power distribution: L＋N＋PE/220Vac/ 220VDC\n\nConverter_Brochure.pdf, trang: 2, trong bảng(figure): None, evidence: Dải điện áp đầu vào từ 85VAC ÷ 300VAC\n\nConverter_Brochure.pdf, trang: 2, trong bảng(figure): None, evidence: Line Frequency: 45 to 65Hz\n\nNetSure_732_User_Manual.pdf, trang: 29, trong bảng(figure): Lightning protection features, evidence: At AC side: The AC input side can withstand five times of simulated lightning voltage of 5Kv at 10/700μs, for the positive and negative polarities respectively. It can withstand five times of simulated lightning surge current of 20Ka at 8/20μs, for the positive and negative polarities respectively.\n\n'],
  'Đầu ra DC': ['5C1A5',
   '09D89',
   '4731B',
   '2188A',
   '656E3',
   'Đáp ứng: 3/5 = 60%',
   'Converter_Brochure.pdf, trang: 2, trong bảng(figure): DC Output Specifications, evidence: Output voltage, Adjustment Range: -42 to -58VDC\n\nConverter_Brochure.pdf, trang: 2, trong bảng(figure): DC Output Specifications, evidence: |- Output voltage, Adjustment Range        | -42 to -58VDC                                                                   |\n\nConverter_Brochure.pdf, trang: 1, evidence: Không có thông tin về độ ổn định điện áp đầu ra trong tài liệu.\n\nNetSure_732_User_Manual.pdf, trang: 28, evidence: Nhiễu (peak-peak) (rated output): ≤200mV（0～20MHz） và Weighted noise (rated output): ≤2mV（300～3400Hz）\n\nNetSure_732_User_Manual.pdf, trang: 28, evidence: Độ  gợn  sóng  đầu ra (đỉnh  –  đỉnh):≤ 200mV（0～20MHz）\n\n'],
  'Yêu cầu với module chỉnh lưu (Rectifier)': ['F916B',
   '8502B',
   '78D05',
   'D0AA4',
   '90A03',
   '66172',
   '88C14',
   'A1813',
   'Đáp ứng: 7/8 = 88%',
   'NetSure_732_Brochure.pdf, trang: 2, trong bảng(figure): DC Output Specifications, evidence: Công suất mỗi module chỉnh lưu: 3000W Maximum\n\nNetSure_732_Brochure.pdf, trang: 1, trong bảng(figure): System Configuration of NetSure™ 732 A41, evidence: Power Factor: ≥0.99(applicable to AC input)\n\nNetSure_732_Brochure.pdf, trang: 1, evidence: Hiệu suất: >95.5%\n\nNetSure_732_User_Manual.pdf, trang: 29, trong bảng(figure): None, evidence: Tổng độ méo hài THD của Rectifier: 4.5% với tải từ 50÷100% tại điện áp 220VAC\n\nNetSure_732_User_Manual.pdf, trang: 25, evidence: Rectifier modules can be inserted or removed with power applied (hot swappable).\n\nNetSure_732_User_Manual.pdf, trang: 29, evidence: Current sharing: The rectifiers can work in parallel and share the current. The unbalanceness is better than ± 5%.\n\nNetSure_732_User_Manual.pdf, trang: 24, evidence: Power Indicator (Green), Protection Indicator (Yellow), Alarm Indicator (Red): The symptoms of usual rectifier faults include: power indicator (green) off, protection indicator (yellow) on, protection indicator blink, fault indicator (red) on and fault indicator blink.\n\nNetSure_732_User_Manual.pdf, trang: 29, evidence: Rectifier fan speed can be set to auto or full speed.\n\n'],
  'Tính năng của thiết bị nguồn': ['B5749',
   '4CC2F',
   '0EE91',
   '0D76B',
   '0649F',
   'F74FE',
   'D701F',
   'Đáp ứng: 6/7 = 86%',
   'Converter_Brochure.pdf, trang: 1, trong bảng(figure): None, evidence: The eSure™ C400/48-3000e3 high-efficiency converter is designed to operate from a nominal 400V DC or 200VAC source to provide nominal -48V DC load power, which is adjustable to application needs. The C400/48-3000e3 is a constant power converter designed with the latest patented switch mode technology.\n\nNetSure_732_User_Manual.pdf, trang: 29, evidence: Lock out at the second over-voltage. When the output voltage reaches the software protection point, the rectifier will shut down, and restart automatically after 5 seconds.\n\nNetSure_732_User_Manual.pdf, trang: 29, trong bảng(figure): None, evidence: LLVD: Default: -44.0 ± 0.2Vdc, configurable through controller（If it is used for outdoor，the default is-－46.6±0.2Vdc）\n\nController_Brochure.pdf, trang: 1, evidence: ...enable remote monitoring and control of modern communication sites.... expanded information and alarm data can be monitored or controlled via password protected and encrypted web browsers... SNMP version 2 or 3...\n\nNetSure_732_User_Manual.pdf, trang: 29, evidence: Mạch bảo vệ quá điện áp và bảo vệ dòng ngắn mạch đã được lắp sẵn trong tủ nguồn.\n\nNetSure_732_User_Manual.pdf, trang: 29, evidence: It can withstand five times of simulated lightning surge current of 20Ka at 8/20μs.\n\nNetSure_732_User_Manual.pdf, trang: 23, trong bảng(figure): Table 4-1 Alarm Handling Methods, evidence: Chương 4 Cách xử lý cảnh báo mô tả các loại cảnh báo như Lỗi AC đầu vào của Rectifier, Lỗi quá nhiệt của Rectifier, Lỗi quạt của Rectifier, Lỗi của Rectifier, Mất điện AC, Cảnh báo AC đầu vào thấp, Cảnh báo AC đầu vào cao, Cảnh báo DC cao, Cảnh báo DC thấp, Cảnh báo LVD.\n\n']
}}

In [55]:
product_key_test

{'Bộ chuyển đổi nguồn 220VAC/ 48VDC (kèm theo 02 dàn acquy 200Ah)': {'Cấu hình thiết bị nguồn': ['96103',
   '277B7',
   '794FC',
   '3ACE9',
   'Đáp ứng: 5/5 = 100%',
   'NetSure_732_User_Manual.pdf, trang: 8, trong bảng(figure): Table 1-1, evidence: Maximum configuration：4 pieces\n\nConverter_Brochure.pdf, trang: 2, trong bảng(figure): None, evidence: Công suất mỗi module chỉnh lưu: 3000W Maximum\n\nNetSure_732_User_Manual.pdf, trang: 8, evidence: Maximum configuration: 4 pieces\n\nNetSure_732_User_Manual.pdf, trang: 10, trong bảng(figure): Table 1-1 Configuration of power system, evidence: Battery MCB: 2×125A/1P; AC output MCB: 1×16A/1P; Load route rated current | Max. output current | Min. cable CSA | Max cable length (volt drop: 0.5V with min. CSA) | Max. cable CSA | Max cable length (volt drop: 0.5V with max. CSA) : 63A | 50A | 16mm2 | 9m | 25mm2 | 14m; 32A | 25A | 10mm2 | 11m | 25mm2 | 29m; 16A | 12A | 6 mm2 | 14m | 25mm2 | 48m.\n\n'],
  'Đầu vào AC': ['992F3',
   'E90A0',
   '2

In [56]:
test

{'96103': {'ten_san_pham': 'Bộ chuyển đổi nguồn 220VAC/ 48VDC (kèm theo 02 dàn acquy 200Ah)',
  'ten_hang_hoa': 'Cấu hình thiết bị nguồn',
  'value': 'Số lượng khe cắm module chỉnh lưu (Rectifier): ≥ 4',
  'yeu_cau_ky_thuat_chi_tiet': 'Số lượng khe cắm module chỉnh lưu (Rectifier): ≥ 4',
  'yeu_cau_ky_thuat': None,
  'query': '"Bộ chuyển đổi nguồn 220VAC/48VDC có cấu hình thiết bị nguồn với số lượng khe cắm module chỉnh lưu (Rectifier) là bao nhiêu? Có phải là ≥ 4 không?"',
  'kha_nang_dap_ung': 'Số lượng khe cắm module chỉnh lưu (Rectifier): 4',
  'tai_lieu_tham_chieu': {'file': 'NetSure_732_User_Manual.pdf',
   'section': 'Table 1-1 Configuration of power system',
   'table_or_figure': 'Table 1-1',
   'page': 8,
   'evidence': 'Maximum configuration：4 pieces'}},
 '277B7': {'ten_san_pham': 'Bộ chuyển đổi nguồn 220VAC/ 48VDC (kèm theo 02 dàn acquy 200Ah)',
  'ten_hang_hoa': 'Cấu hình thiết bị nguồn',
  'value': 'Công suất mỗi module chỉnh lưu: ≥ 3000W',
  'yeu_cau_ky_thuat_chi_tiet': '

In [127]:
!pip install python-docx

In [58]:
from docx import Document
from docx.shared import Pt
from docx.enum.table import WD_TABLE_ALIGNMENT, WD_ALIGN_VERTICAL
# === Tạo Word document ===
doc = Document()
doc.add_heading("BẢNG TUYÊN BỐ ĐÁP ỨNG VỀ KỸ THUẬT", level=1)

# Tạo bảng 6 cột
table = doc.add_table(rows=1, cols=6)
table.style = 'Table Grid'
table.alignment = WD_TABLE_ALIGNMENT.CENTER

# Header
headers = [
    "Hạng mục số", "Tên hàng hoá",
    "Thông số kỹ thuật và các tiêu chuẩn của hàng hoá trong E-HSMT",
    "Thông số kỹ thuật và các tiêu chuẩn của hàng hoá chào trong E-HSDT",
    "Hồ sơ tham chiếu", "Tình đáp ứng của hàng hoá"
]

for i, text in enumerate(headers):
    cell = table.rows[0].cells[i]
    cell.text = text
    for p in cell.paragraphs:
        for run in p.runs:
            run.font.bold = True
            run.font.size = Pt(10)
    cell.vertical_alignment = WD_ALIGN_VERTICAL.CENTER

# Ghi từng dòng
for product, hang_hoa_dict in product_key_test.items():
    for idx, (ten_hang_hoa, items) in enumerate(hang_hoa_dict.items(), start=1):
        ma_ids = items[:-2]  # Các ID
        dap_ung = items[-2]  # ví dụ: "đáp ứng"
        ho_so = items[-1]    # tài liệu tham chiếu

        # Tổng hợp thông số kỹ thuật
        eh_smt = ""
        eh_hsdt = ""

        for ma in ma_ids:
            if ma in test:
                eh_smt += f"- {test[ma]['yeu_cau_ky_thuat_chi_tiet']}\n"
                eh_hsdt += f"- {test[ma].get('kha_nang_dap_ung',"")}\n"

        # Tạo dòng mới
        row = table.add_row().cells
        row[0].text = str(idx)
        row[1].text = ten_hang_hoa
        row[2].text = eh_smt.strip()
        row[3].text = eh_hsdt.strip()
        row[4].text = ho_so
        row[5].text = dap_ung

        for cell in row:
            cell.vertical_alignment = WD_ALIGN_VERTICAL.TOP
            for p in cell.paragraphs:
                for run in p.runs:
                    run.font.size = Pt(9)

# Lưu file
doc.save("bang_tuyen_bo_dap_ung2.docx")